# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt

In [62]:
region_1 = pd.read_csv('/datasets/geo_data_0.csv')
region_2 = pd.read_csv('/datasets/geo_data_1.csv')
region_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [63]:
def regions_info(df):
    display(df.head())
    display(df.describe())
    display(df.info())
    display('Количество дупликатов', df.duplicated().sum())

In [64]:
for i in [region_1, region_2, region_3]:
    regions_info(i)
    

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

'Количество дупликатов'

...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

'Количество дупликатов'

0

In [65]:
display(region_1.corr())
display(region_2.corr())
display(region_3.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


### Вывод

Данные по регионам были загружены в переменные region_1, region_2 и region_3. Пропуски в данных обнаружены не были.

## Обучение и проверка модели

### region_1

In [66]:
X = region_1.drop(['product', 'id'], axis=1)
y = region_1['product']

In [67]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [68]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions_1 = pd.Series(model.predict(X_val))
rmse_1 = mean_squared_error(y_val, predictions_1) ** 0.5
mean_1 = predictions_1.mean()
true_1 = y_val.reset_index(drop=True)
predictions_1.reset_index(drop=True)
print('RMSE для региона 1 равен', rmse_1)
print('среднее значение для региона 1 равно', mean_1)

RMSE для региона 1 равен 37.75660035026169
среднее значение для региона 1 равно 92.39879990657768


### region_2

In [69]:
X = region_2.drop(['product', 'id'], axis=1)
y = region_2['product']

In [70]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [71]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions_2 = pd.Series(model.predict(X_val))
rmse_2 = mean_squared_error(y_val, predictions_2) ** 0.5
mean_2 = predictions_2.mean()
true_2 = y_val.reset_index(drop=True)
predictions_2.reset_index(drop=True)
print('RMSE для региона 2 равен', rmse_2)
print('среднее значение для региона 2 равно', mean_2)

RMSE для региона 2 равен 0.890280100102884
среднее значение для региона 2 равно 68.71287803913762


### region_3

In [72]:
X = region_3.drop(['product', 'id'], axis=1)
y = region_3['product']

In [73]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

In [74]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions_3 = pd.Series(model.predict(X_val))
rmse_3 = mean_squared_error(y_val, predictions_3) ** 0.5
mean_3 = predictions_3.mean()
true_3 = y_val.reset_index(drop=True)
predictions_3.reset_index(drop=True)
print('RMSE для региона 3 равен', rmse_3)
print('среднее значение для региона 3 равно', mean_3)

RMSE для региона 3 равен 40.145872311342174
среднее значение для региона 3 равно 94.77102387765939


### Вывод

Для каждого региона была создана и обучена модель. Модели регионов 1 и 3 имеют схожие средние значения предсказаний(92.4 и 94.8), а также схоже высокую rmse(37.7 и 40.1). Модель региона 2 имеет приемлимо низкую rmse, но среднее значение - ниже предыдущих(68.1)

## Подготовка к расчёту прибыли

In [75]:
BUDGET = 10000000000
REV_PER_PRODUCT = 450000
WELLS_NUMBER = 500
BEST_WELLS_NUMBER = 200

In [76]:
min_product = BUDGET / (REV_PER_PRODUCT * BEST_WELLS_NUMBER)
min_product

111.11111111111111

In [77]:
display(true_1.mean())
display(true_2.mean())
true_3.mean()

92.32595637084387

68.72538074722745

95.15099907171961

In [78]:
def wells_revenue(true, pred):
    top_predictions = pred.sort_values(ascending=False)
    wells_data = true[top_predictions.index][:BEST_WELLS_NUMBER]  
    return wells_data.sum() * REV_PER_PRODUCT - BUDGET

### Вывод

Был подсчитан минимальный средний объем сырья для безубыточной разработки новой скважины - 111.1. Данное значение ниже средних запасов во всех регионах, что подтверждает необходимось в выборе только лучших скважин. 

## Расчёт прибыли и рисков 

**Изменения** 

Доверительные интервалы для всех регионов были перерассчитаны: 0.025 квантиль для верхней границы, 0.975 квантиль - для нижней.  

In [79]:
state = np.random.RandomState(12345)

### region_1

In [80]:
revenues_1 = []
for i in range(1000):
    pred_subsample = predictions_1.sample(n=WELLS_NUMBER, replace=True, random_state=state)
    true_subsample = true_1[pred_subsample.index]
    revenues_1.append(wells_revenue(true_subsample, pred_subsample))

In [81]:
revenues_1 = pd.Series(revenues_1)
display(revenues_1.mean())

435933772.1386621

In [83]:
print(
        f"95% доверительный интервал для региона 1 лежит между: \
{revenues_1.quantile(0.025):.0f} руб. и {revenues_1.quantile(0.975):.0f} руб."
    )

95% доверительный интервал для региона 1 лежит между: -116231613 руб. и 966504181 руб.


In [84]:
((revenues_1 < 0).sum() / len(revenues_1)) * 100

6.1

### region_2

In [85]:
revenues_2 = []
for i in range(1000):
    pred_subsample = predictions_2.sample(n=WELLS_NUMBER, replace=True, random_state=state)
    true_subsample = true_2[pred_subsample.index]
    revenues_2.append(wells_revenue(true_subsample, pred_subsample))

In [86]:
revenues_2 = pd.Series(revenues_2)
display(revenues_2.mean())

498569016.91678834

In [87]:
print(
        f"95% доверительный интервал для региона 2 лежит между: \
{revenues_2.quantile(0.025):.0f} руб. и {revenues_2.quantile(0.975):.0f} руб."
    )

95% доверительный интервал для региона 2 лежит между: 75315573 руб. и 911503695 руб.


In [88]:
((revenues_2 < 0).sum() / len(revenues_2)) * 100

1.0

### region_3

In [89]:
revenues_3 = []
for i in range(1000):
    pred_subsample = predictions_3.sample(n=WELLS_NUMBER, replace=True, random_state=state)
    true_subsample = true_3[pred_subsample.index]
    revenues_3.append(wells_revenue(true_subsample, pred_subsample))

In [90]:
revenues_3 = pd.Series(revenues_3)
display(revenues_3.mean())

412672898.98853195

In [91]:
print(
        f"95% доверительный интервал для региона 3 лежит между: \
{revenues_3.quantile(0.025):.0f} руб. и {revenues_3.quantile(0.975):.0f} руб."
    )

95% доверительный интервал для региона 3 лежит между: -158939516 руб. и 978191202 руб.


In [92]:
((revenues_3 < 0).sum() / len(revenues_3)) * 100

7.000000000000001

### Вывод

Были рассчитаны средняя прибыль, доверительные интервалы и риски убытков для всех трех регионов:

* Первый регион:
  * средняя прибыль - 435933772 руб;
  *  95%-й доверительный интервал - -116231613, 966504181;
  * риск убытков - 6.1%.
  
  
* Второй регион:
  * средняя прибыль - 498569017 руб;
  *  95%-й доверительный интервал - 75315573, 911503695;
  * риск убытков - 1.0%.  
  
  
* Третий регион:
  * средняя прибыль - 412672898 руб;
  *  95%-й доверительный интервал - -158939516, 978191202;
  * риск убытков - 7.0%.
  
  
Исходя из этих данных, а также из требования о максимальном риске убытков 2.5%, наиболее подходящий регион для разработки - **второй**.